In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cervical_Cancer"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Cervical_Cancer/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Cervical_Cancer/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Cervical_Cancer/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Cervical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. Find the most relevant directory for Cervical Cancer
subdirectories = os.listdir(tcga_root_dir)
target_trait = trait.lower().replace("_", " ")  # Convert to lowercase for case-insensitive matching

# Search for related terms to Cervical Cancer
related_terms = ["cervical", "cervix", "cesc"]
matched_dir = None

for subdir in subdirectories:
    subdir_lower = subdir.lower()
    # Check if any related term is in the directory name
    if any(term in subdir_lower for term in related_terms):
        matched_dir = subdir
        print(f"Found potential match: {subdir}")
        # If exact match found, select it
        if "cervical_cancer" in subdir_lower.replace(" ", "_"):
            print(f"Selected as best match: {subdir}")
            matched_dir = subdir
            break

# If we found a potential match, use it
if matched_dir:
    print(f"Selected directory: {matched_dir}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, matched_dir)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not matched_dir:
    print("Skipping this trait as no suitable data was found.")


Found potential match: TCGA_Cervical_Cancer_(CESC)
Selected as best match: TCGA_Cervical_Cancer_(CESC)
Selected directory: TCGA_Cervical_Cancer_(CESC)
Clinical file: TCGA.CESC.sampleMap_CESC_clinicalMatrix
Genetic file: TCGA.CESC.sampleMap_HiSeqV2_PANCAN.gz



Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_treatment_completion_success_outcome', 'adjuvant_rad_therapy_prior_admin', 'age_at_initial_pathologic_diagnosis', 'age_began_smoking_in_years', 'agent_total_dose_count', 'assessment_timepoint_category', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'birth_control_pill_history_usage_category', 'brachytherapy_administered_status', 'brachytherapy_first_reference_point_administered_total_dose', 'brachytherapy_method_other_specify_text', 'brachytherapy_method_type', 'cervical_carcinoma_corpus_uteri_involvement_indicator', 'cervical_carcinoma_pelvic_extension_text', 'cervical_neoplasm_pathologic_margin_involved_text', 'cervical_neoplasm_pathologic_margin_involved_type', 'chemotherapy_negation_radiation_therapy_concurrent_adminstrd_txt', 'chemotherapy_negation_radiation_therapy_concurrnt_nt_

### Step 2: Find Candidate Demographic Features

In [3]:
# 1. Identify candidate columns for age and gender
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, "TCGA_Cervical_Cancer_(CESC)"))
clinical_data = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Candidate columns for age and gender
candidate_age_cols = [
    'age_at_initial_pathologic_diagnosis', 
    'age_began_smoking_in_years', 
    'days_to_birth'
]
candidate_gender_cols = ['gender']

# 2. Preview the candidate columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_data.columns:
        age_preview[col] = clinical_data[col].head(5).tolist()

gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_data.columns:
        gender_preview[col] = clinical_data[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [51.0, 31.0, 53.0, 48.0, 49.0], 'age_began_smoking_in_years': [nan, nan, 22.0, nan, nan], 'days_to_birth': [-18886.0, -11611.0, -19473.0, -17839.0, -18215.0]}

Gender columns preview:
{'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine age columns
print("Selecting demographic columns for age and gender information:")

age_cols_info = {
    'age_at_initial_pathologic_diagnosis': "Contains direct age values",
    'age_began_smoking_in_years': "Contains age when patient started smoking (too many NaN values)",
    'days_to_birth': "Contains negative days (days from birth to diagnosis date)"
}

for col, desc in age_cols_info.items():
    print(f"- {col}: {desc}")

# Select the most appropriate age column
# 'age_at_initial_pathologic_diagnosis' is the best choice as it directly contains patient age
age_col = 'age_at_initial_pathologic_diagnosis'

# Examine gender columns
gender_cols_info = {
    'gender': "Contains direct gender information (FEMALE/MALE)"
}

for col, desc in gender_cols_info.items():
    print(f"- {col}: {desc}")

# Select the gender column
gender_col = 'gender'

# Print the selected columns
print("\nSelected demographic columns:")
print(f"Age column: {age_col}")
print(f"Gender column: {gender_col}")


Selecting demographic columns for age and gender information:
- age_at_initial_pathologic_diagnosis: Contains direct age values
- age_began_smoking_in_years: Contains age when patient started smoking (too many NaN values)
- days_to_birth: Contains negative days (days from birth to diagnosis date)
- gender: Contains direct gender information (FEMALE/MALE)

Selected demographic columns:
Age column: age_at_initial_pathologic_diagnosis
Gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Cervical_Cancer/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 308)
Clinical data saved to ../../output/preprocess/Cervical_Cancer/clinical_data/TCGA.csv
Clinical data shape: (313, 3)
Number of samples in clinical data: 313
Number of samples in genetic data: 308
Number of common samples: 308
Linked data shape: (308, 19851)


Data shape after handling missing values: (308, 19851)
For the feature 'Cervical_Cancer', the least common label is '0' with 3 occurrences. This represents 0.97% of the dataset.
The distribution of the feature 'Cervical_Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 38.0
  50% (Median): 46.0
  75%: 56.0
Min: 20.0
Max: 88.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 308 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Dataset deemed not usable based on validation criteria. Data not saved.
Preprocessing completed.
